In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision
!pip install torchvision

In [ ]:
!pip install torch torchvision torchmetrics tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 46.3 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm

In [ ]:
import os
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

# 载入CIFAR-10数据的工具函数
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

# 自定义CIFAR-10数据集类
class CIFAR10Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.data = []
        self.labels = []

        # 加载训练数据
        for i in range(1, 6):  # data_batch_1 to data_batch_5
            batch_file = os.path.join(data_dir, f'data_batch_{i}')
            batch_dict = unpickle(batch_file)
            self.data.append(batch_dict[b'data'])
            self.labels.append(batch_dict[b'labels'])

        # 合并所有的数据和标签
        self.data = np.concatenate(self.data)
        self.labels = np.concatenate(self.labels)

        # 将数据转换为32x32x3的图像格式
        self.data = self.data.reshape((self.data.shape[0], 3, 32, 32))
        self.data = self.data.transpose(0, 2, 3, 1)  # 转换为 (batch_size, height, width, channels)

        # 标签转为tensor
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# 图像预处理
data_transform = {
    "train": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
    "val": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
}

# 设置数据路径
train_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/cifar-10/cifar-10-batches-py"
val_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/cifar-10/cifar-10-batches-py"  # 可以调整为测试数据集路径

# 加载数据集
train_dataset = CIFAR10Dataset(data_dir=train_dir, transform=data_transform["train"])
val_dataset = CIFAR10Dataset(data_dir=val_dir, transform=data_transform["val"])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# 输出数据形状
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break


torch.Size([16, 3, 224, 224]) torch.Size([16])


In [ ]:
!pip install torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 13.8 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from torchmetrics.classification import MulticlassAccuracy, MulticlassAUROC
import torch.nn.functional as F  # 确保导入 F 模块

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 数据加载工具函数
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

# 自定义CIFAR-10数据集类
class CIFAR10Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.data = []
        self.labels = []

        # 加载数据
        for i in range(1, 6):
            batch_file = os.path.join(data_dir, f'data_batch_{i}')
            batch_dict = unpickle(batch_file)
            self.data.append(batch_dict[b'data'])
            self.labels.append(batch_dict[b'labels'])

        # 合并数据和标签
        self.data = np.concatenate(self.data)
        self.labels = np.concatenate(self.labels)
        self.data = self.data.reshape((self.data.shape[0], 3, 32, 32)).transpose(0, 2, 3, 1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# 图像预处理
data_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 加载数据集
train_dir = train_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/cifar-10/cifar-10-batches-py"
train_dataset = CIFAR10Dataset(data_dir=train_dir, transform=data_transform)
val_dataset = CIFAR10Dataset(data_dir=train_dir, transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# 定义GoogLeNet模型
model = models.googlenet(weights='IMAGENET1K_V1', aux_logits=True)
model.fc = nn.Linear(1024, 10)
if model.aux1:
    model.aux1.fc2 = nn.Linear(model.aux1.fc2.in_features, 10)
if model.aux2:
    model.aux2.fc2 = nn.Linear(model.aux2.fc2.in_features, 10)
model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_and_evaluate(model, train_loader, val_loader, epochs):
    accuracy_metric = MulticlassAccuracy(num_classes=10).to(device)
    auroc_metric = MulticlassAUROC(num_classes=10).to(device)

    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_loss, train_correct = 0, 0
        y_true_train, y_pred_train, y_prob_train = [], [], []
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            if isinstance(outputs, tuple):
                logits, aux1, aux2 = outputs
                loss = criterion(logits, labels) + 0.3 * criterion(aux1, labels) + 0.3 * criterion(aux2, labels)
            else:
                logits = outputs
                loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            train_correct += (predicted == labels).sum().item()

            y_true_train.extend(labels.cpu().numpy())
            y_pred_train.extend(predicted.cpu().numpy())

            # 获取概率分布用于 AUC 计算
            probs = F.softmax(logits, dim=1).detach().cpu().numpy()
            y_prob_train.extend(probs)

        train_acc = train_correct / len(train_loader.dataset)
        train_loss /= len(train_loader)
        train_precision = precision_score(y_true_train, y_pred_train, average='macro')
        train_recall = recall_score(y_true_train, y_pred_train, average='macro')
        train_f1 = f1_score(y_true_train, y_pred_train, average='macro')
        train_roc_auc = roc_auc_score(y_true_train, y_prob_train, multi_class='ovr')

        print(f"\n[Train] Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, "
              f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, "
              f"F1 Score: {train_f1:.4f}, AUC-ROC: {train_roc_auc:.4f}")

        # 验证阶段
        model.eval()
        val_loss, val_correct = 0, 0
        y_true_val, y_pred_val, y_prob_val = [], [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

                if isinstance(outputs, tuple):
                    logits, aux1, aux2 = outputs
                    loss = criterion(logits, labels) + 0.3 * criterion(aux1, labels) + 0.3 * criterion(aux2, labels)
                else:
                    logits = outputs
                    loss = criterion(logits, labels)

                val_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                val_correct += (predicted == labels).sum().item()

                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(predicted.cpu().numpy())

                # 获取概率分布用于 AUC 计算
                probs = F.softmax(logits, dim=1).detach().cpu().numpy()
                y_prob_val.extend(probs)

        val_acc = val_correct / len(val_loader.dataset)
        val_loss /= len(val_loader)
        val_precision = precision_score(y_true_val, y_pred_val, average='macro')
        val_recall = recall_score(y_true_val, y_pred_val, average='macro')
        val_f1 = f1_score(y_true_val, y_pred_val, average='macro')
        val_roc_auc = roc_auc_score(y_true_val, y_prob_val, multi_class='ovr')

        print(f"[Validation] Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, "
              f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, "
              f"F1 Score: {val_f1:.4f}, AUC-ROC: {val_roc_auc:.4f}")


Using device: cpu


In [ ]:
train_and_evaluate(model, train_loader, val_loader, epochs=10)

Epoch 1/10 - Training: 100%|██████████| 1563/1563 [16:28<00:00,  1.58it/s]



[Train] Loss: 1.0973, Accuracy: 0.7981, Precision: 0.7977, Recall: 0.7981, F1 Score: 0.7978, AUC-ROC: 0.9777


Epoch 1/10 - Validation: 100%|██████████| 1563/1563 [04:43<00:00,  5.51it/s]


[Validation] Loss: 0.4296, Accuracy: 0.8533, Precision: 0.8646, Recall: 0.8533, F1 Score: 0.8544, AUC-ROC: 0.9904


Epoch 2/10 - Training: 100%|██████████| 1563/1563 [16:34<00:00,  1.57it/s]


...

[Validation] Loss: 0.0399, Accuracy: 0.9873, Precision: 0.9874, Recall: 0.9873, F1 Score: 0.9873, AUC-ROC: 0.9999


Epoch 10/10 - Training: 100%|██████████| 1563/1563 [16:41<00:00,  1.56it/s]



[Train] Loss: 0.1591, Accuracy: 0.9759, Precision: 0.9759, Recall: 0.9759, F1 Score: 0.9759, AUC-ROC: 0.9996


Epoch 10/10 - Validation: 100%|██████████| 1563/1563 [04:26<00:00,  5.86it/s]


[Validation] Loss: 0.0548, Accuracy: 0.9824, Precision: 0.9831, Recall: 0.9824, F1 Score: 0.9826, AUC-ROC: 0.9998
